In [5]:
from astropy.io.votable import parse_single_table    
from astropy.table import Table

from connectors.tapquery import AsyncJob  
    """
    Asynchronous job class

    Class which allows a user to log into GACS and submit an
    asynchronous query via the TAP+ interface.
    For now, we only do AsyncJobs as part of the TAP+ standard.
    """

import numpy as np
import os

import pandas_profiling
import pandas as pd

import tempfile

In [6]:
def get_gaia_data(query, target):
    
    """
    Query a TAP service (designated by its tap_endpoint)
    with a given ADQL query
    
    Query is performed asynchronously
    
    Return an AstroPy Table object
    
    This object is converted to a PandasDataframe
    """
    
    async_check_interval = 1
    gacs_tap_conn = AsyncJob(target, query, poll_interval=async_check_interval)

    # Run the job (start + wait + raise_exception)
    gacs_tap_conn.run()

    # Store the response
    result = gacs_tap_conn.open_result()

    tmp_vot = tempfile.NamedTemporaryFile(delete = False)
    with open(tmp_vot.name, 'w') as f:
        f.write(result.content.decode("utf-8"))

    table = parse_single_table(tmp_vot.name).to_table()

    # finally delete temp files
    os.unlink(tmp_vot.name)
    
    #returns the pandas dataframe
    return pd.DataFrame(np.ma.filled(table.as_array()), columns=table.colnames)

In [7]:
query = """
        SELECT source_id, ra, dec, phot_g_mean_flux, phot_g_mean_mag,
        DISTANCE(POINT('ICRS',ra,dec), POINT('ICRS',266.41683,-29.00781)) 
        AS dist FROM gaiadr1.gaia_source WHERE 1=CONTAINS(POINT('ICRS',ra,dec),
        CIRCLE('ICRS',266.41683,-29.00781, 0.08333333))
        """
# sample query from https://gea.esac.esa.int/archive/

target = "http://gea.esac.esa.int/tap-server/tap"



df = get_gaia_data(query, target)

gaiamagcols=['dec', 'dist', 'phot_g_mean_flux', 'phot_g_mean_mag', 'ra', 'source_id']
gaiadf = df[gaiamagcols]
pandas_profiling.ProfileReport(gaiadf)

Number of variables,6
Number of observations,1316
Total Missing (%),0.0%
Total size in memory,61.8 KiB
Average record size in memory,48.1 B
Numeric,6
Categorical,0
Date,0
Text (Unique),0
Rejected,0
Distinct count,1316
